In [73]:
import plotly.graph_objects as go

import pandas as pd
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from matplotlib import colors as mcolors
import numpy as np
import os


In [74]:
# 1. Cargar datos
df = pd.read_csv(r"C:\Users\nono_\AppRegresion\data\autos_limpios.csv")
print(df.columns)


Index(['marca', 'modelo', 'ano', 'color', 'tipo de combustible', 'puertas',
       'transmision', 'motor', 'tipo de carroceria', 'fecha publicacion',
       'kilometros', 'precio', 'traccion'],
      dtype='object')


In [75]:
import os
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

# 1) Tu DataFrame original
# df = pd.read_csv(...)

# 2) Paleta cualitativa
palette = px.colors.qualitative.Plotly

# 3) Lista ordenada de marcas
marcas = sorted(df['marca'].unique())

# 4) Construcción de trazas stacked: UNA traza POR MARCA
traces = []
for marca in marcas:
    sub = (
        df[df['marca'] == marca]
          .groupby('modelo')
          .size()
          .reset_index(name='n')
          .sort_values('n', ascending=False)
    )
    colors = [palette[i % len(palette)] for i in range(len(sub))]
    traces.append(go.Bar(
        x=[marca] * len(sub),
        y=sub['n'],
        name=marca,               # Cada traza se nombra con la marca
        marker_color=colors,
        customdata=sub['modelo'],
        hovertemplate=(
            "<b>Marca:</b> %{x}<br>"
            "<b>Modelo:</b> %{customdata}<br>"
            "<b>Muestras:</b> %{y}<extra></extra>"
        )
    ))

fig = go.Figure(data=traces)

# 5) Layout y leyenda interactiva
fig.update_layout(
    barmode='stack',
    title="Número de muestras por Marca (apilado por Modelo)",
    xaxis_title="Marca",
    yaxis_title="Número de muestras",
    showlegend=True,  
    legend=dict(
        title="Marcas",
        itemclick="toggle",        
        itemdoubleclick="toggleothers"
    ),
    autosize=True,
    xaxis=dict(
        tickangle=-45,
        automargin=True
    ),
    margin=dict(t=60, b=80, l=20, r=20)
)


# 6) Mostrar en notebook (opcional)
# fig.show()

# 7) Guardado del HTML
ruta_guardado   = "C:/Users/nono_/AppRegresion/static/plots"
nombre_archivo  = "numero_muestras_marcas.html"
os.makedirs(ruta_guardado, exist_ok=True)

fig.write_html(
    os.path.join(ruta_guardado, nombre_archivo),
    include_plotlyjs='cdn',       # Plotly.js desde CDN
    full_html=True,               # Página completa
    config={
        'responsive': True,       # activa redibujo en resize
        'modeBarButtonsToRemove': [
            'sendDataToCloud',
            'lasso2d',
            'select2d',
            'zoomIn2d',
            'zoomOut2d',
            'toggleSpikelines',
            'hoverCompareCartesian',
            'hoverClosestCartesian',
            'resetScale2d',
            'pan2d',
        ],
        'displaylogo': False
    }
)
fig.show()

In [76]:
import os
import pandas as pd
import plotly.graph_objects as go

# Normalizar nombres
df['marca'] = df['marca'].str.lower().str.strip()

# Agrupar por marca y año para obtener el precio promedio
df_grouped = df.groupby(['marca', 'ano'])['precio'].mean().reset_index()

# Obtener marcas únicas
marcas = df_grouped['marca'].unique()

# Crear figura
fig = go.Figure()

# Agregar una traza por cada marca
for marca in marcas:
    df_marca = df_grouped[df_grouped['marca'] == marca]
    fig.add_trace(go.Scatter(
        x=df_marca['ano'],
        y=df_marca['precio'],
        mode='lines+markers',
        name=marca.capitalize()
    ))

# Layout final con responsive y márgenes
fig.update_layout(
    title="Precio medio por año para cada marca",
    xaxis_title="Año",
    yaxis_title="Precio promedio",
    xaxis=dict(autorange="reversed"),
    yaxis=dict(autorange=True),
    # Estos dos para que sea responsive dentro del iframe
    autosize=True,
    margin=dict(t=40, b=20, l=20, r=20)
)

# Ruta y nombre del archivo
ruta_guardado = "C:/Users/nono_/AppRegresion/static/plots"
nombre_archivo = "precio_medio_marcas.html"
os.makedirs(ruta_guardado, exist_ok=True)

# Exportar a HTML ligero y responsive
fig.write_html(
    os.path.join(ruta_guardado, nombre_archivo),
    include_plotlyjs='cdn',    # Plotly.js desde CDN
    full_html=True,            # HTML completo con cabecera
    config={
        'responsive': True,    # redibuja en resize
        'modeBarButtonsToRemove': [
            'sendDataToCloud',
            'lasso2d',
            'select2d',
            'zoomIn2d',
            'zoomOut2d',
            'toggleSpikelines',
            'hoverCompareCartesian',
            'hoverClosestCartesian',
            'resetScale2d',
            'pan2d',
        ],
        'displaylogo': False
    }
)

# (Opcional) Mostrar en notebook o consola
fig.show()




In [77]:
import plotly.express as px
import numpy as np
import os

# Copiar columna
df[["año"]] = df[["ano"]]

# Calcular correlación
corr = df[["precio", "kilometros", "año", "puertas"]].corr()

# Crear figura
fig = px.imshow(corr, text_auto=True, title="Mapa de correlaciones")

# Ruta de guardado
ruta_guardado = "C:/Users/nono_/AppRegresion/static/plots"
nombre_archivo = "correlacion_mapa.html"
os.makedirs(ruta_guardado, exist_ok=True)

# Guardar gráfico como HTML
fig.write_html(
    os.path.join(ruta_guardado, nombre_archivo),
    config={
        'modeBarButtonsToRemove': [
            'sendDataToCloud',
            'lasso2d',
            'select2d',
            'zoomIn2d',
            'zoomOut2d',
            'toggleSpikelines',
            'hoverCompareCartesian',
            'hoverClosestCartesian',
            'resetScale2d',
            'pan2d',
        ],
        'displaylogo': False
    },
    include_plotlyjs='cdn'
)

# Mostrar gráfico en Jupyter o VSCode si estás trabajando allí
fig.show()


In [78]:
import os
import plotly.express as px

# 1) Construcción del gráfico
fig = px.violin(
    df,
    x="marca",
    y="precio",
    box=True,
    color="marca",
    title="Distribución de precios según marca"
)

# Ajustes estéticos y de eje
fig.update_xaxes(tickangle=30)

# 2) Hacer responsive y ajustar márgenes
fig.update_layout(
    autosize=True,
    margin=dict(t=40, b=20, l=20, r=20)
)

# 3) Mostrar en notebook o entorno interactivo (opcional)
# fig.show()

# 4) Definir ruta de guardado
ruta_guardado = "C:/Users/nono_/AppRegresion/static/plots"
os.makedirs(ruta_guardado, exist_ok=True)

# 5) Guardar como HTML ligero y responsive
fig.write_html(
    os.path.join(ruta_guardado, "violin_precios_marcas.html"),
    include_plotlyjs='cdn',   # Plotly.js desde CDN
    full_html=True,           # HTML completo con cabecera
    config={
        'responsive': True,   # redibuja al cambiar tamaño
        'modeBarButtonsToRemove': [
            'sendDataToCloud', 'lasso2d', 'select2d',
            'zoomIn2d', 'zoomOut2d', 'toggleSpikelines',
            'hoverCompareCartesian', 'hoverClosestCartesian',
            'resetScale2d', 'pan2d'
        ],
        'displaylogo': False
    }
)

fig.show()


In [128]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from matplotlib import colors as mcolors
import numpy as np
import os

# Paso 1: Crear nodo raíz (marcas)
df_brand = df.groupby('marca')['precio'].mean().reset_index()
df_brand['id'] = df_brand['marca']
df_brand['parent'] = ""
df_brand['value'] = df_brand['precio']
df_brand['label'] = df_brand['marca']

# Paso 2: Crear nodo intermedio (modelos)
df_model = df.groupby(['marca', 'modelo'])['precio'].mean().reset_index()
df_model['id'] = df_model['marca'] + ' - ' + df_model['modelo']
df_model['parent'] = df_model['marca']
df_model['value'] = df_model['precio']
df_model['label'] = df_model['modelo']

# Paso 3: Crear nodo hoja (años)
df['modelo_full'] = df['marca'] + ' - ' + df['modelo']
df_year = df.groupby(['modelo_full', 'ano'])['precio'].mean().reset_index()
df_year['id'] = df_year['modelo_full'] + ' - ' + df_year['ano'].astype(str)
df_year['parent'] = df_year['modelo_full']
df_year['value'] = df_year['precio']
df_year['label'] = df_year['ano'].astype(str)

# Función para convertir 'rgb(r, g, b)' a '#rrggbb'
def rgb_str_to_hex(rgb_str):
    parts = rgb_str.strip('rgb()').split(',')
    r, g, b = [int(p) for p in parts]
    return '#{:02x}{:02x}{:02x}'.format(r, g, b)

# Aclara color base (mezcla con blanco)
def lighten_color(color, amount=0.5):
    if isinstance(color, str) and color.startswith('rgb'):
        color = rgb_str_to_hex(color)
    c = mcolors.to_rgb(color)
    white = np.array([1, 1, 1])
    mixed = np.array(c) * (1 - amount) + white * amount
    return mcolors.to_hex(mixed)

# Orden jerárquico personalizado
df_brand_sorted = df_brand.sort_values('value', ascending=False)
ordered = []

for _, b in df_brand_sorted.iterrows():
    ordered.append(b)

    df_mod_sorted = df_model[df_model['parent'] == b['id']].sort_values('value', ascending=False)
    for _, m in df_mod_sorted.iterrows():
        ordered.append(m)

        df_yrs = df_year[df_year['parent'] == m['id']].copy()
        df_yrs['ano_int'] = df_yrs['id'].str.extract(r' - (\d{4})$').astype(int)
        df_yrs_sorted = df_yrs.sort_values('ano_int', ascending=False)

        for _, y in df_yrs_sorted.iterrows():
            ordered.append(y)

nodes_ordered = pd.DataFrame(ordered)

# Crear colores
unique_brands = nodes_ordered[nodes_ordered['parent'].isna() | (nodes_ordered['parent'] == "")]['id'].unique()
palette = px.colors.qualitative.Plotly
brand_colors = {brand: palette[i % len(palette)] for i, brand in enumerate(unique_brands)}

colors_final = []

for _, row in nodes_ordered.iterrows():
    if pd.isna(row['parent']) or row['parent'] == "":
        color = brand_colors.get(row['id'], '#777777')
        colors_final.append(color)
    else:
        # Subnodos: obtener marca raíz
        parent_id = row['parent']
        marca_id = None
        while True:
            parent_row = nodes_ordered[nodes_ordered['id'] == parent_id]
            if parent_row.empty:
                break
            parent_row = parent_row.iloc[0]
            if pd.isna(parent_row['parent']) or parent_row['parent'] == "":
                marca_id = parent_row['id']
                break
            parent_id = parent_row['parent']

        base_color = brand_colors.get(marca_id, '#777777')
        parent_row = nodes_ordered[nodes_ordered['id'] == row['parent']].iloc[0]
        if pd.isna(parent_row['parent']) or parent_row['parent'] == "":
            color = lighten_color(base_color, amount=0.3)
        else:
            color = lighten_color(base_color, amount=0.6)

        colors_final.append(color)

nodes_ordered['color'] = colors_final

# EXTRA: construir texto personalizado para el hover
def get_hover_text(row):
    parts = row['id'].split(' - ')
    if len(parts) == 1:
        return f"<b>Marca:</b> {parts[0]}<br>Precio promedio: ${row['value']:,.0f}"
    elif len(parts) == 2:
        return f"<b>Marca:</b> {parts[0]}<br><b>Modelo:</b> {parts[1]}<br>Precio promedio: ${row['value']:,.0f}"
    else:
        return f"<b>Marca:</b> {parts[0]}<br><b>Modelo:</b> {parts[1]}<br><b>Año:</b> {parts[2]}<br>Precio promedio: ${row['value']:,.0f}"

nodes_ordered['hover'] = nodes_ordered.apply(get_hover_text, axis=1)

fig = go.Figure(go.Treemap(
    labels       = nodes_ordered['label'],     # Solo muestra marca, modelo o año
    parents      = nodes_ordered['parent'],
    ids          = nodes_ordered['id'],
    values       = nodes_ordered['value'],
    marker_colors= nodes_ordered['color'],
    sort         = False,                      # Respeta el orden original
    textinfo     = "label",
    hovertext    = nodes_ordered['hover'],
    hoverinfo    = "text"
))


# 1) Ajustes de layout para que sea responsive
fig.update_layout(
    title="Precio promedio descendente agrupado por marca → modelo → año",
    autosize=True,
   
    margin=dict(t=40, b=20, l=20, r=20)
)

ruta_guardado   = "C:/Users/nono_/AppRegresion/static/plots"
nombre_archivo = "treemap_marcas_modelos.html"
# 3) Guardar como HTML ligero y responsive
fig.write_html(
    os.path.join(ruta_guardado, nombre_archivo),
    include_plotlyjs="https://cdn.plot.ly/plotly-3.0.1.min.js",   # Plotly.js desde CDN
    full_html=False,           # HTML completo con cabecera
    config={
        'responsive': True,   # fuerza redibujo al cambiar tamaño
        'modeBarButtonsToRemove': [
            'sendDataToCloud', 'lasso2d', 'select2d',
            'zoomIn2d', 'zoomOut2d', 'toggleSpikelines',
            'hoverCompareCartesian', 'hoverClosestCartesian',
            'resetScale2d', 'pan2d'
        ],
        'displaylogo': False
    }
)
fig.show()

In [80]:
import plotly.graph_objs as go

# Suponemos que estas son tus columnas: ['modelo', 'anio', 'precio']
modelos = df['modelo'].unique()

# Lista para guardar trazas (una por modelo)
trazas = []
for i, modelo in enumerate(modelos):
    datos = df[df['modelo'] == modelo]
    traza = go.Scatter(
        x=datos['ano'],
        y=datos['precio'],
        mode='markers',
        name=modelo,
        visible=(i == 0)  # Solo la primera visible al inicio
    )
    trazas.append(traza)

botones = []
for i, modelo in enumerate(modelos):
    visibilidad = [False] * len(modelos)
    visibilidad[i] = True

    datos_modelo = df[df['modelo'] == modelo]
    x_min = datos_modelo['ano'].min()
    x_max = datos_modelo['ano'].max()

    botones.append(dict(
        label=modelo,
        method='update',
        args=[
            {'visible': visibilidad},
            {
                'title': {
                    'text': f'Precio por Año - {modelo}',
                    'x': 0.5,
                    'xanchor': 'center'
                },
                'xaxis': {
                    'range': [x_min - 1, x_max + 1],
                    'tickmode': 'linear',
                    'dtick': 1,
                    'title': 'Año'
                }
            }
        ]
    ))


# Crear figura
fig = go.Figure(data=trazas)
fig.update_layout(
    title=dict(
        text=f'Precio por Año y modelo:  {modelos[0]}',
        x=0.5,
        xanchor='center'
    ),
    xaxis_title='Año',
    yaxis_title='Precio',
    updatemenus=[{
        'buttons': botones,
        'direction': 'down',
        'showactive': True,
        'x': 0.1,
        'y': 1.2
    }],
    template='ggplot2'
)



ruta_guardado = "C:/Users/nono_/AppRegresion/static/plots"
nombre_archivo = "precio_modelo_selectivo.html"
fig.write_html(
    os.path.join(ruta_guardado, nombre_archivo),
    config={
        'responsive': True, 
        'modeBarButtonsToRemove': [
            'sendDataToCloud',
            'lasso2d',
            'select2d',
            'zoomIn2d',
            'zoomOut2d',
            'toggleSpikelines',
            'hoverCompareCartesian',
            'hoverClosestCartesian',
            'resetScale2d',
            'pan2d',
        ],
        'displaylogo': False
    },
    include_plotlyjs='cdn'
)
fig.show()